In [2]:
# https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
analyzer = SentimentIntensityAnalyzer()

## Helper functions

In [3]:
# the columns are beeing merged togheter here
def merge_columns_in_dataframe(df):
    columns_to_merge = ['page_title', 'sub_title', 'introduction', 'summary_box', 'content', 'accordion']
    new_df = df.loc[:, ["url", "page_title"]].copy()
    new_df['text'] = df.apply(lambda row: ' '.join([str(row[col]) for col in columns_to_merge]), axis=1)
    return new_df

In [4]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german") # eventuell anderen stemmer verwenden
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

In [5]:

def get_sentiment(text_string):
    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

def add_sentiment_cols(dataframe, text_col="text"):
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [6]:
def get_analysis_data(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = merge_columns_in_dataframe(filtered_df)
    data["text"] = data["text"].apply(preprocess_text)
    return add_sentiment_cols(data)

In [7]:
def save_as_json(pandas_dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    pandas_dataframe.to_json(f"{file_name}_{date_info}.json", orient='records')

In [14]:
df = get_analysis_data("ratgeber_pages_part2.json")
save_as_json(df, "ratgeber_pages_with_rating")

## Execution of Code

In [16]:
df2 = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_2.json")
# save_as_json(df2, "full_scrape_2_analysis")

In [76]:
df2 = pd.read_json("full_scrape_2_analysis_15-03-24_15:40:08.json")
display(df2)

,url,page_title,text,neg,neu,pos,compound,judgement
0,https://www.mobiliar.ch/versicherungen-und-vor...,Tierversicherung,tierversicher hund katz versich tierversicher ...,0.000,0.994,0.006,0.2406,positive
1,https://www.mobiliar.ch/versicherungen-und-vor...,Cyberversicherung,cyberversicher hilf digital alltag bankgeschaf...,0.028,0.954,0.019,-0.5362,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Wertsachenversicherung,"wertsachenversicher versich , bedeutet wertvol...",0.000,0.981,0.019,0.6553,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Die Hausratversicherung,hausratversicher rundum-versicher beweg sach w...,0.021,0.979,0.000,-0.6662,negative
4,https://www.mobiliar.ch/versicherungen-und-vor...,Mietkautionsversicherung,mietkautionsversicher dank mietkaution spar mi...,0.007,0.976,0.017,0.4329,positive
...,...,...,...,...,...,...,...,...
1456,https://www.mobiliar.ch/versicherungen-und-vor...,Die Fahrrad-Kaskoversicherung vermitteln,fahrrad-kaskoversicher vermitteln umfass versi...,0.028,0.972,0.000,-0.3313,negative
1457,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2022: Insgesamt 19 weitere Brücken un...,projekt 2022 : insgesamt 19 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1458,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2023: Insgesamt 13 weitere Brücken un...,projekt 2023 : insgesamt 13 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1459,https://www.mobiliar.ch/versicherungen-und-vor...,Winter Wanderland,"wint wanderland verschneit tal , weit nebelme ...",0.000,1.000,0.000,0.0000,neutral


In [82]:
df2["judgement"].value_counts()

judgement
neutral     726
positive    465
negative    270
Name: count, dtype: int64

## to display the Functions

In [15]:
json_file_example = [{"url": "https://www.mobiliar.ch/die-mobiliar/ueber-uns/unsere-werbung/werbespots-liebe-mobiliar/liebe-mobiliar-parkett", "page_title": "Liebe Mobiliar: Parkett", "sub_title": "", "introduction": "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.", "summary_box": "", "content": "Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert.", "accordion": ""},]

In [20]:
pandas_dataframe = pd.DataFrame(json_file_example)
pandas_dataframe

,url,page_title,sub_title,introduction,summary_box,content,accordion
0,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Liebe Mobiliar: Parkett,,"Der neue «Liebe Mobiliar»-Spot zeigt, dass Sch...",,Die Mobiliar startet mit dem neusten «Liebe Mo...,


In [25]:
merged_text_df = merge_columns_in_dataframe(pandas_dataframe)
for i in merged_text_df["text"]:
    print (i)

Liebe Mobiliar: Parkett  Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert. 


In [51]:
text = "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert."

In [69]:
text = "Fahren fuhren gefahren fuhr die Fahrt er fährt"

In [70]:
tokens = word_tokenize(text.lower())
tokens

['fahren', 'fuhren', 'gefahren', 'fuhr', 'die', 'fahrt', 'er', 'fährt']

In [71]:
filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
filtered_tokens


['fahren', 'fuhren', 'gefahren', 'fuhr', 'fahrt', 'fährt']

In [72]:
# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german") # eventuell anderen stemmer verwenden
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
stemmed_tokens

['fahr', 'fuhr', 'gefahr', 'fuhr', 'fahrt', 'fahrt']

In [73]:
from nltk.stem import WordNetLemmatizer
# from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
lemmatized_tokens

['fahren', 'fuhren', 'gefahren', 'fuhr', 'fahrt', 'fährt']

In [68]:
nlp = spacy.load("de_core_news_sm")

# Beispieltext
text = "fahren fuhren gefahren fuhr die Fahrt"

# Verarbeiten des Textes mit SpaCy
doc = nlp(text)

filtered_tokens = [
        token for token in doc if token not in stopwords.words("german")
    ]
# Lemmatisieren der Tokens
lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
lemmatized_tokens

['fahren', 'Fuhr', 'fahren', 'fahren', 'der', 'Fahrt']